# COGS 108 - Data Checkpoint

# Names

- Mariam Bachar (A16217374)
- Alexandra Hernandez (A16730685)
- Brian Kwon (A16306826)
- Andrew Uhm (A16729684)
- Ethan Wang (A17229824)

<a id='research_question'></a>
# Research Question

*Do certain keywords as identified by CLIP correlate with the popularity (as measured by the equivalent of “likes”) that artwork receives on social media?*

# Dataset(s)

*Fill in your dataset information here*

(Copy this information for each dataset)
- Dataset Name:
- Link to the dataset:
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [47]:
import requests
import urllib
from bs4 import BeautifulSoup
import deviantart
from datetime import datetime
import time

#please use your own client id and client secret to getting all of us IP banned lol. Shouldn't happen but might.
da = deviantart.Api("25492", "06217cf59e73b401dc0a14d00857a793")


for i in range(0,1):
    deviations = da.browse(endpoint='popular', offset=i*10, limit=10)['results']

    for deviation in deviations:
        url = deviation.content['src']
        dId = deviation.deviationid
        urllib.request.urlretrieve(url, f"images/{dId}.png")
        
    time.sleep(15) #sleep 15 seconds to avoid deviantart throttling us.
    

#Possible things to collect:
#-----------------------------
#url = deviation.content['src']
#deviation.title
#deviation.author
#deviation.deviationid
#deviation.stats['favorites']
#deviation.stats['comments']
#deviation.url
#dt = datetime.fromtimestamp(int(dailydeviations[0].published_time)))

https://www.deviantart.com/cryptid-creations/art/3244-Mimikyu-Fan-Art-958059784
https://www.deviantart.com/tc-96/art/Mother-s-Day-2023-962405128
https://www.deviantart.com/cryptid-creations/art/3245-Light-Jolteon-TCG-Fan-Art-958059861
https://www.deviantart.com/apofiss/art/sneaky-bud-962602943
https://www.deviantart.com/loulouvz/art/Moushley-Fanart-961968477
https://www.deviantart.com/rongs1234/art/Splash-pose-962047244
https://www.deviantart.com/outcastcomix/art/Happy-Mother-s-Day-2023-962447151
https://www.deviantart.com/senchafox/art/Lavender-Dreams-Adoptable-961901277
https://www.deviantart.com/neldorwen/art/Commission-Yor-Forger-961879310
https://www.deviantart.com/anatofinnstark/art/The-Eclipse-Knight-962145757


# Data Cleaning

Describe your data cleaning steps here.

In [3]:
# code (can have multiple)